In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from river import drift
from itertools import product
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
''' Leitura do arquivo com as funções necessárias pro pré-processamento e processamento do dados '''
# json_processed_data(folder_path), gpx_processed_data(data_folder_path) e fit_processed_data(data_folder_path)
%run C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/final/functions.ipynb

In [ ]:
''' Contador de arquivos dentro da pasta '''

# Caminho da pasta com as pastas contendo as atividades de cada atleta
data_folders_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities'

folders_count = 0

for file in os.listdir(data_folders_path):
    folders_count += 1

print(folders_count)

In [ ]:
# Criação do Dataframe que terá os valores de cada atleta por id
df_athletes_activities = pd.DataFrame()

# Número do primeiro id (Se começar com 0, mudar para n = 0)
n = 1

# Iteração para criar o Dataframe com os valores de cada atleta por id
for athlete_id in range(n, folders_count + n):

    # Nome do arquivo com os dados do atleta
    athlete_foldername = f'athlete{athlete_id}'

    # Junção da pasta com as pastas contendo as atividades de cada atleta e o nome da pasta com os dados do atleta por id
    data_folder_path = os.path.join(data_folders_path, athlete_foldername)

    # Pegando a primeira atividade da pasta
    first_file = os.listdir(data_folder_path)[0]
    
    if first_file.endswith('.gpx'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = gpx_processed_data(data_folder_path)

    elif first_file.endswith('.json'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = json_processed_data(data_folder_path)

    elif first_file.endswith('.fit'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = fit_processed_data(data_folder_path)
    
    df_athlete_activity = activities_time_frequency(df)

    # Adição de uma coluna athlete_id para identificar os dados de cada atleta
    df_athlete_activity.insert(0, 'athlete_id', athlete_id)

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities = pd.concat([df_athletes_activities, df_athlete_activity])

df_athletes_activities.reset_index(drop=True, inplace=True)

In [ ]:
df_athletes_activities

In [ ]:
sns.set_style('darkgrid')
g = sns.countplot(df_athletes_activities, hue="activity_type", x='athlete_id')
g.legend().set_title(None)
plt.xlabel('Values by Athlete id')
plt.ylabel('Days')

In [ ]:
df_athletes_activities_new = df_athletes_activities[['athlete_id', 'activity_date', 'activity_type','days_between_activities']]

df_athletes_activities_new['activity_type'] = (df_athletes_activities_new['activity_type'] == 'Activity') * 1
df_athletes_activities_new

In [ ]:
# Quantidade de atividades num período específico de dias
activity_days = 7

# Somando o número de atividades dentro desse período de dias em uma janela deslizante, por atleta
df_athletes_activities_new['activities_moving_sum'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['activity_type'].rolling(activity_days).sum()]

df_athletes_activities_new

In [ ]:
# Dicionário para adicionar os exemplos
examples_dict = []

# Lista com o id de cada atleta
athlete_ids = df_athletes_activities_new['athlete_id'].unique()

# Quantidade de atividades para estar dentro do período específico
activities_qtd = 5

# Iteração a partir do id do atleta
for athlete_id in athlete_ids:
    # Inicializando com periodo ativo como Falso
    active_period = False
    # Atribuindo a uma variável o dataframe filtrado por id do atleta
    df_athlete = df_athletes_activities_new.loc[df_athletes_activities_new['athlete_id'] == athlete_id]

    # Iteração pelas linhas com a série de cada atleta por id
    for index, row in df_athlete.iterrows():
        # Condição para caso a soma de atividades no periodo específico seja >= activities_qtd, se inicializa o periodo ativo
        if (row['activities_moving_sum'] >= activities_qtd) & (active_period == False):
            active_period = True
            active_period_dict = {'athlete_id': athlete_id, 'start': row['activity_date']}

        # Condição para finalizar o período ativo caso a soma de atividade no período ativo seja igual 0
        elif (row['activities_moving_sum'] == 0) & (active_period == True):
            active_period_dict['end'] = row['activity_date']
            active_period_dict['data'] = df_athlete[(df_athlete['activity_date'] >= active_period_dict['start']) & (df_athlete['activity_date'] <= active_period_dict['end'])].reset_index(drop=True)
            active_period = False    
            examples_dict.append(active_period_dict)

In [ ]:
# Dataframe com os dados dos atletas que tiveram inicio e fim de períodos ativos para compor exemplos
df_examples = pd.DataFrame(examples_dict)
df_examples

In [ ]:
df_examples_train, df_examples_test = train_test_split(df_examples, test_size=0.2, random_state=123)
df_examples_train.reset_index(drop=True, inplace=True)
df_examples_test.reset_index(drop=True, inplace=True)

In [ ]:
df_examples_train

In [ ]:
df_examples_test

In [ ]:
''' ADWIN '''
adwin_params_dict = {'delta': [0.0001, 0.005, 0.001, 0.05, 0.01],
                     'min_window_length': [5, 10, 20, 30, 40],
                     'clock': [1, 2, 3, 4, 5],
                     'max_buckets': [5, 10, 15, 20, 25],
                     'grace_period': [5, 10, 15, 20, 25]}

''' KSWIN '''
kswin_params_dict = {'alpha': [0.001, 0.005, 0.01, 0.05, 0.1],
                     'window_size':[100, 200, 300, 400, 500],
                     'stat_size': [5, 10, 15, 20, 25],
                     'seed': [321]}

''' Page Hinkley '''
ph_params_dict = {'delta': [0.0001, 0.005, 0.001, 0.05, 0.01],
                  'min_instances': [30, 50, 100, 200, 300],
                  'threshold': [1.0, 2.0, 3.0, 4.0, 5.0],
                  'alpha': [0.5, 1.0, 1.5, 2.0, 2.5]}


products = [['ADWIN', [dict(zip(('delta', 'min_window_length', 'clock', 'max_buckets', 'grace_period'), (i,j,k,w,z))) for i,j,k,w,z in product(adwin_params_dict['delta'], adwin_params_dict['min_window_length'], adwin_params_dict['clock'], adwin_params_dict['max_buckets'], adwin_params_dict['grace_period'])]],
            ['KSWIN', [dict(zip(('alpha', 'window_size', 'stat_size', 'seed'), (i,j,k,w))) for i,j,k,w in product(kswin_params_dict['alpha'], kswin_params_dict['window_size'], kswin_params_dict['stat_size'], kswin_params_dict['seed'])]],
            ['PageHinkley', [dict(zip(('delta', 'min_instances', 'threshold', 'alpha'), (i,j,k,w))) for i,j,k,w in product(ph_params_dict['delta'], ph_params_dict['min_instances'], ph_params_dict['threshold'], ph_params_dict['alpha'])]]]

In [ ]:
len(products[0][1]) + len(products[1][1]) + len(products[2][1])

In [ ]:
# Valor para criar o range de drifts como TP
x = 3

# Contador para cada paramêtro utilizado nos métodos para cada exemplo
param_combination_id = 0

# Contador para identicar o exemplo processado
example_id = 0

# Performance resultada com a função de detecção de drift
train_results = []

# Iteração pela lista products contendo na primeira posição o nome do método e na segunda as combinações de parâmetros
for drift_method, params in products:
    # Iteração pela combinação de parâmetros para trabalhar com elas separadamente                   
    for param in params:
        # Iteração por cada df contendo um exemplo
        for df_example_train in df_examples_train['data']:
            # Condição para detectar drift com as combinações de parâmetros caso o método seja ADWIN
            if drift_method == 'ADWIN':
                drift_detection(df_example_train, drift.ADWIN(**param), train_results, param, drift_method, x, param_combination_id, example_id) 

            # Condição para detectar drift com as combinações de parâmetros caso o método seja KSWIN
            elif drift_method == 'KSWIN':
                drift_detection(df_example_train, drift.KSWIN(**param), train_results, param, drift_method, x, param_combination_id, example_id)
                
            # Condição para detectar drift com as combinações de parâmetros caso o método seja Page Hinkley
            elif drift_method == 'PageHinkley':
                drift_detection(df_example_train, drift.PageHinkley(**param), train_results, param, drift_method, x, param_combination_id, example_id)
            
            example_id += 1

        param_combination_id += 1

In [ ]:
df_results_train = pd.DataFrame(train_results)
df_results_train

In [ ]:
df_results_train_new = df_results_train.groupby('Parameter Combination id').agg({'TP': 'sum', 'TN': 'sum', 'FP': 'sum', 'FN': 'sum'})
df_results_train_new

In [ ]:
for index, row in df_results_train_new.iterrows():
    
    df_results_train_new.loc[index, 'Precision'] = metrics(row['TP'], row['FP'], row['TN'], row['FN'])[0]
    df_results_train_new.loc[index, 'Recall'] = metrics(row['TP'], row['FP'], row['TN'], row['FN'])[1]
    df_results_train_new.loc[index, 'F1-score'] = metrics(row['TP'], row['FP'], row['TN'], row['FN'])[2]
 
df_results_train_metrics = df_results_train_new.reset_index()
df_results_train_metrics

In [ ]:
df_results_train_metrics[['FN', 'FP', 'TP']].describe()

In [ ]:
df_results_train_metrics[df_results_train_metrics['F1-score'] == df_results_train_metrics['F1-score'].max()]